# Panther Library Demo: BERT Optimization
In this demo, we will optimize a BERT model's linear layers using `panther`. We will benchmark memory, speed, and accuracy (MLM loss) before and after tuning.

## 0. Install Dependencies
Install the required libraries for the demo.

In [1]:
%pip install transformers datasets install rich

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for install


## 1. Setup and Imports
Importing necessary libraries and setting up the device (GPU/CPU) and the seed.

In [2]:
import time
import copy
import torch
import numpy as np
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import BertForMaskedLM, BertTokenizer, logging
import pickle
from typing import Any, Dict, List, Optional
from rich import print

# Import panther components
from panther.tuner import (
    SKAutoTuner,
    LayerConfig,
    TuningConfigs,
    OptunaSearch,
    SearchAlgorithm,
)

# Setting up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"Random seed set to {seed} for reproducibility")


set_seed(42)

d:\Coding Projects\panther\panther_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda

Random seed set to 42 for reproducibility

## 2. Utility Functions
Helper functions for model inspection, benchmarking (time/memory), and the Dataset class.

In [3]:
def count_parameters(model):
    """Count trainable parameters in the model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def model_size_info(model):
    """Get detailed size information about the model"""
    total_params = count_parameters(model)
    layer_params = {}

    # Check BERT layers
    if hasattr(model, "bert") and hasattr(model.bert, "encoder"):
        for i, layer in enumerate(model.bert.encoder.layer):
            layer_params[f"bert.encoder.layer.{i}"] = sum(
                p.numel() for p in layer.parameters() if p.requires_grad
            )

    # Check MLM head
    if hasattr(model, "cls"):
        if hasattr(model.cls, "predictions"):
            if hasattr(model.cls.predictions, "transform"):
                layer_params["cls.predictions.transform"] = sum(
                    p.numel()
                    for p in model.cls.predictions.transform.parameters()
                    if p.requires_grad
                )
            if hasattr(model.cls.predictions, "decoder"):
                layer_params["cls.predictions.decoder"] = sum(
                    p.numel()
                    for p in model.cls.predictions.decoder.parameters()
                    if p.requires_grad
                )

    return {
        "total_params": total_params,
        "total_params_millions": total_params / 1e6,
        "layer_params": layer_params,
    }


def measure_time_with_stats(func, *args, n_runs=1, warmup=1):
    """Measure execution time with GPU synchronization"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

    for _ in range(warmup):
        result = func(*args)
        if torch.cuda.is_available():
            torch.cuda.synchronize()

    times = []
    for _ in range(n_runs):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        start = time.time()
        result = func(*args)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        end = time.time()
        times.append(end - start)

    times = np.array(times)
    mean_time = np.mean(times)
    std_time = np.std(times)

    return {
        "mean": mean_time,
        "std": std_time,
        "min": np.min(times),
        "max": np.max(times),
        "samples_per_sec": 1.0 / mean_time,
        "samples_per_sec_std": std_time / (mean_time * mean_time),
    }


def evaluate_model_with_stats(model, dataloader, tokenizer=None, n_runs=1):
    """Evaluate model loss with multiple runs"""
    all_results = []
    for run in range(n_runs):
        model.eval()
        total_loss = 0
        total_samples = 0
        with torch.no_grad():
            for batch in dataloader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)
                loss = outputs.loss
                batch_size = batch["input_ids"].size(0)
                total_loss += loss.item() * batch_size
                total_samples += batch_size
        avg_loss = total_loss / total_samples
        all_results.append({"loss": avg_loss})

    losses = [res["loss"] for res in all_results]
    return {
        "loss_mean": np.mean(losses),
        "loss_std": np.std(losses),
        "runs": all_results,
    }


class MaskedTextDataset(Dataset):
    """Dataset for masked language modeling"""

    def __init__(self, texts, tokenizer, max_length=128, seed=42):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.seed = seed

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            return_special_tokens_mask=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        input_ids = encoding.input_ids.clone().squeeze(0)
        special_tokens_mask = encoding.special_tokens_mask.squeeze(0).bool()
        labels = input_ids.clone()
        mask_positions = (~special_tokens_mask).nonzero(as_tuple=True)[0]
        num_to_mask = max(1, int(0.15 * len(mask_positions)))

        mask_indices = np.random.choice(
            mask_positions.tolist(), size=num_to_mask, replace=False
        )

        input_ids[mask_indices] = self.tokenizer.mask_token_id
        attention_mask = encoding.attention_mask.squeeze(0)
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }


# implementing custom gridsearch. serves as an example of custom search logic and is optimal here given the small search space (9 combinations).
class GridSearch(SearchAlgorithm):
    """
    Grid search algorithm that systematically tries all combinations of parameters up to a maximum number of iterations.

    This implementation performs a search through a pre-generated grid of parameter
    combinations. It iterates through these combinations and keeps track of the best
    parameters found and their corresponding score.

    Attributes:
        max_iterations: The maximum number of parameter combinations to try.
        param_space: Dictionary mapping parameter names to lists of their possible values.
        indexed_param_space: A list of dictionaries, where each dictionary represents a unique
                             combination of parameter values.
        curr_iteration: The current iteration number, indicating which parameter combination
                        is being evaluated or will be evaluated next.
        best_score: The highest score achieved so far during the search.
        best_params: The dictionary of parameters that achieved the best_score.
    """

    def __init__(self, max_iterations: int = 10):
        """
        Initialize the GridSearch algorithm.

        Args:
            max_iterations: The maximum number of iterations to run.
        """
        self.max_iterations = max_iterations
        self.param_space: Dict[str, List] = {}
        self.curr_iteration: int = 0
        self.indexed_param_space: List[Dict[str, Any]] = []
        self.history: List[Dict[str, Any]] = []
        self.best_params: Optional[Dict[str, Any]] = None
        self.best_score: float = -float("inf")
        self.reset()

    @property
    def current_idx(self):
        """Alias for curr_iteration for backward compatibility."""
        return self.curr_iteration

    @current_idx.setter
    def current_idx(self, value):
        self.curr_iteration = value

    @property
    def param_combinations(self):
        """Alias for indexed_param_space for backward compatibility."""
        return self.indexed_param_space

    @param_combinations.setter
    def param_combinations(self, value):
        self.indexed_param_space = value

    def initialize(self, param_space: Dict[str, List]):
        """
        Initialize the search algorithm with the parameter space.

        Args:
            param_space: Dictionary of parameter names and their possible values
        """
        self.reset()
        self.param_space = param_space
        self.indexed_param_space = self._generate_indexed_param_space()
        self.max_iterations = min(self.max_iterations, len(self.indexed_param_space))

    def _my_product(self, value_lists: List[List[Any]]) -> List[tuple[Any, ...]]:
        """
        Computes the Cartesian product of a list of lists.
        Example: _my_product([[1, 2], ['a', 'b']]) -> [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]
        """
        if not value_lists:
            return [()]

        product_tuples: List[tuple[Any, ...]] = [()]

        for current_list_values in value_lists:
            if not current_list_values:
                return []

            product_tuples = [
                existing_tuple + (item,)
                for existing_tuple in product_tuples
                for item in current_list_values
            ]

        return product_tuples

    def _generate_indexed_param_space(self):
        """
        Generates an indexed parameter space from the provided param_space.
        The indexed parameter space is a list of dictionaries, where each dictionary
        represents a unique combination of parameter values.

        Raises:
            ValueError: If param_space is None.

        Returns:
            List[Dict[str, Any]]: The indexed parameter space.
        """
        if self.param_space is None:
            raise ValueError("param_space is None")

        param_names = list(self.param_space.keys())
        value_lists = list(self.param_space.values())

        values_combined = self._my_product(value_lists)
        indexed_param_space = [{} for _ in range(len(values_combined))]

        i = 0
        for value_combination in values_combined:
            param = {}
            j = 0
            for value in value_combination:
                param[param_names[j]] = value
                j = j + 1
            indexed_param_space[i] = param
            i = i + 1

        return indexed_param_space

    def get_next_params(self) -> Optional[Dict[str, Any]]:
        """
        Get the next set of parameters to try.

        Returns:
            Dictionary of parameter names and values to try, or None if finished
        """
        if self.is_finished():
            return None

        params = self.indexed_param_space[self.curr_iteration]
        self.curr_iteration = self.curr_iteration + 1
        return params

    def update(self, params: Dict[str, Any], score: float):
        """
        Update the search algorithm with the results of the latest trial.

        Args:
            params: Dictionary of parameter names and values that were tried
            score: The evaluation score for the parameters
        """
        self.history.append({"params": params, "score": score})
        if self.best_score < score:
            self.best_score = score
            self.best_params = params

    def save_state(self, filepath: str):
        """
        Save the current state of the search algorithm to a file.

        Args:
            filepath: The path to the file where the state should be saved.
        """
        state = {
            "param_space": self.param_space,
            "curr_iteration": self.curr_iteration,
            "indexed_param_space": self.indexed_param_space,
            "history": self.history,
            "best_params": self.best_params,
            "best_score": self.best_score,
        }

        with open(filepath, "wb") as f:
            pickle.dump(state, f)

    def load_state(self, filepath: str):
        """
        Load the state of the search algorithm from a file.

        Args:
            filepath: The path to the file from which the state should be loaded.
        """
        with open(filepath, "rb") as f:
            state = pickle.load(f)

        self.param_space = state["param_space"]
        self.curr_iteration = state["curr_iteration"]
        self.indexed_param_space = state["indexed_param_space"]
        self.history = state.get("history", [])
        self.best_params = state["best_params"]
        self.best_score = state["best_score"]

    def get_best_params(self) -> Optional[Dict[str, Any]]:
        """
        Get the best set of parameters found so far.

        Returns:
            Dictionary of the best parameter names and values, or None if no params yet.
        """
        return self.best_params

    def get_best_score(self) -> Optional[float]:
        """
        Get the best score achieved so far.

        Returns:
            The best score, or None if no score yet.
        """
        return self.best_score

    def reset(self):
        """
        Reset the search algorithm to its initial state while preserving param_space.
        If param_space is set, it regenerates the combinations.
        """
        # Preserve param_space if it exists
        preserved_param_space = getattr(self, "param_space", {})

        # Reset all state
        self.param_space = {}
        self.curr_iteration = 0
        self.indexed_param_space = []
        self.history = []
        self.best_params = None
        self.best_score = -float("inf")

        # Restore and regenerate if param_space was set
        if preserved_param_space:
            self.param_space = preserved_param_space
            self.indexed_param_space = self._generate_indexed_param_space()

    def is_finished(self) -> bool:
        """
        Check if the search algorithm has finished its search (e.g., budget exhausted).

        Returns:
            True if the search is finished, False otherwise.
        """
        return self.curr_iteration == self.max_iterations


## 3. Data Preparation
Loading the WikiText dataset and preparing the dataloader.

In [4]:
def get_data():
    print("Preparing BERT test dataset...")
    try:
        from datasets import load_dataset

        num_examples = 100
        print("Loading WikiText dataset for evaluation...")
        wiki_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

        texts = []
        for item in wiki_dataset:
            text = item["text"].strip()
            if len(text) >= 50 and len(text.split()) > 10:
                texts.append(text)

        if len(texts) < num_examples:
            print(
                f"Found only {len(texts)} examples in test split, loading more from train..."
            )
            wiki_train = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
            for item in wiki_train:
                text = item["text"].strip()
                if len(text) >= 50 and len(text.split()) > 10:
                    texts.append(text)
                    if len(texts) >= num_examples + 50:
                        break
        print(f"Loaded {len(texts)} examples from WikiText dataset")
    except Exception as e:
        raise RuntimeError(f"Failed to load WikiText dataset: {str(e)}")

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    dataset = MaskedTextDataset(texts, tokenizer)
    batch_size = 16
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    memory_batch = {k: v.to(device) for k, v in next(iter(dataloader)).items()}
    return tokenizer, dataloader, memory_batch


# Execute data loading
tokenizer, val_loader, memory_batch = get_data()

Preparing BERT test dataset...

Loading WikiText dataset for evaluation...

Loaded 1937 examples from WikiText dataset

## 4. Model Initialization
Loading and prepare BERT-base.

In [5]:
# Initialize models
logging.set_verbosity_error()
model = BertForMaskedLM.from_pretrained("bert-base-uncased").to(device)
logging.set_verbosity_warning()
model.eval()
orig_model = copy.deepcopy(model)

# Model stats
orig_params = model_size_info(model)

# Align vocab size to multiple of 16 for Tensor Cores
orig_out_features = model.cls.predictions.decoder.weight.size(0)
new_out_features = ((orig_out_features + 15) // 16) * 16


# Wrapper for post-processing logic
def post_process_outputs(model_outputs, orig_size=orig_out_features):
    if hasattr(model_outputs, "logits") and model_outputs.logits is not None:
        if model_outputs.logits.size(-1) > orig_size:
            model_outputs.logits = model_outputs.logits[..., :orig_size]
    return model_outputs


def create_wrapped_forward(original_forward_fn):
    def wrapped_forward(*args, **kwargs):
        outputs = original_forward_fn(*args, **kwargs)
        return post_process_outputs(outputs)

    return wrapped_forward


# Capture original forward methods
true_orig_forward = model.forward
true_orig_ref_forward = orig_model.forward

# Apply modifications to both models
for m in [model, orig_model]:
    if orig_out_features != new_out_features:
        orig_weight = m.cls.predictions.decoder.weight
        orig_bias = m.cls.predictions.decoder.bias
        new_weight = torch.zeros(
            new_out_features,
            orig_weight.size(1),
            device=orig_weight.device,
            dtype=orig_weight.dtype,
        )
        new_bias = torch.zeros(
            new_out_features, device=orig_bias.device, dtype=orig_bias.dtype
        )
        new_weight[:orig_out_features, :] = orig_weight
        new_bias[:orig_out_features] = orig_bias
        new_decoder = torch.nn.Linear(orig_weight.size(1), new_out_features, bias=True)
        new_decoder.weight = torch.nn.Parameter(new_weight)
        new_decoder.bias = torch.nn.Parameter(new_bias)
        m.cls.predictions.decoder = new_decoder
    m.config.vocab_size = new_out_features

model.forward = create_wrapped_forward(true_orig_forward)
orig_model.forward = create_wrapped_forward(true_orig_ref_forward)

## 5. Configure Panther AutoTuner
Defining the evaluation functions and tuning configuration for the MLM head linear layers.

In [6]:
# Define accuracy function (negative loss)
def acc_eval_func(model):
    # results = evaluate_model_with_stats(model, val_loader, tokenizer) # uncomment this for real accuracy
    results = {
        "loss_mean": 10  # dummy for demo
    }
    return -results["loss_mean"]


# Define function to maxmize
def speed_eval_func(model):
    return -count_parameters(model)


loss_threshold = -9999  # Allow loose threshold for demo

# Configure tuner
configs = TuningConfigs(
    [
        LayerConfig(
            layer_names={"type": "Linear"},
            params="auto",
            separate=False,
        ),
    ]
)

tuner = SKAutoTuner(
    model=copy.deepcopy(model),
    configs=configs,
    accuracy_eval_func=acc_eval_func,
    accuracy_threshold=loss_threshold,
    optmization_eval_func=speed_eval_func,
    verbose=True,
    search_algorithm=GridSearch(max_iterations=10),
    num_runs_per_param=1,
)

In [7]:
print("configs resolved from 'auto':", tuner.configs)

configs resolved from 'auto': 
TuningConfigs(configs=[LayerConfig(layer_names=['bert.encoder.layer.0.attention.output.dense', 
'bert.encoder.layer.0.attention.self.key', 'bert.encoder.layer.0.attention.self.query', 
'bert.encoder.layer.0.attention.self.value', 'bert.encoder.layer.1.attention.output.dense', 
'bert.encoder.layer.1.attention.self.key', 'bert.encoder.layer.1.attention.self.query', 
'bert.encoder.layer.1.attention.self.value', 'bert.encoder.layer.10.attention.output.dense', 
'bert.encoder.layer.10.attention.self.key', 'bert.encoder.layer.10.attention.self.query', 
'bert.encoder.layer.10.attention.self.value', 'bert.encoder.layer.11.attention.output.dense', 
'bert.encoder.layer.11.attention.self.key', 'bert.encoder.layer.11.attention.self.query', 
'bert.encoder.layer.11.attention.self.value', 'bert.encoder.layer.2.attention.output.dense', 
'bert.encoder.layer.2.attention.self.key', 'bert.encoder.layer.2.attention.self.query', 
'bert.encoder.layer.2.attention.self.value', 'bert.encoder.layer.3.attention.output.dense', 
'bert.encoder.layer.3.attention.self.key', 'bert.encoder.layer.3.attention.self.query', 
'bert.encoder.layer.3.attention.self.value', 'bert.encoder.layer.4.attention.output.dense', 
'bert.encoder.layer.4.attention.self.key', 'bert.encoder.layer.4.attention.self.query', 
'bert.encoder.layer.4.attention.self.value', 'bert.encoder.layer.5.attention.output.dense', 
'bert.encoder.layer.5.attention.self.key', 'bert.encoder.layer.5.attention.self.query', 
'bert.encoder.layer.5.attention.self.value', 'bert.encoder.layer.6.attention.output.dense', 
'bert.encoder.layer.6.attention.self.key', 'bert.encoder.layer.6.attention.self.query', 
'bert.encoder.layer.6.attention.self.value', 'bert.encoder.layer.7.attention.output.dense', 
'bert.encoder.layer.7.attention.self.key', 'bert.encoder.layer.7.attention.self.query', 
'bert.encoder.layer.7.attention.self.value', 'bert.encoder.layer.8.attention.output.dense', 
'bert.encoder.layer.8.attention.self.key', 'bert.encoder.layer.8.attention.self.query', 
'bert.encoder.layer.8.attention.self.value', 'bert.encoder.layer.9.attention.output.dense', 
'bert.encoder.layer.9.attention.self.key', 'bert.encoder.layer.9.attention.self.query', 
'bert.encoder.layer.9.attention.self.value', 'cls.predictions.transform.dense'], params={'num_terms': [1, 2, 3], 
'low_rank': [16, 32, 64, 128]}, separate=False, copy_weights=True), 
LayerConfig(layer_names=['bert.encoder.layer.0.intermediate.dense', 'bert.encoder.layer.1.intermediate.dense', 
'bert.encoder.layer.10.intermediate.dense', 'bert.encoder.layer.11.intermediate.dense', 
'bert.encoder.layer.2.intermediate.dense', 'bert.encoder.layer.3.intermediate.dense', 
'bert.encoder.layer.4.intermediate.dense', 'bert.encoder.layer.5.intermediate.dense', 
'bert.encoder.layer.6.intermediate.dense', 'bert.encoder.layer.7.intermediate.dense', 
'bert.encoder.layer.8.intermediate.dense', 'bert.encoder.layer.9.intermediate.dense'], params={'num_terms': [1, 2, 
3], 'low_rank': [256, 32, 64, 128, 16]}, separate=False, copy_weights=True), 
LayerConfig(layer_names=['bert.encoder.layer.0.output.dense', 'bert.encoder.layer.1.output.dense', 
'bert.encoder.layer.10.output.dense', 'bert.encoder.layer.11.output.dense', 'bert.encoder.layer.2.output.dense', 
'bert.encoder.layer.3.output.dense', 'bert.encoder.layer.4.output.dense', 'bert.encoder.layer.5.output.dense', 
'bert.encoder.layer.6.output.dense', 'bert.encoder.layer.7.output.dense', 'bert.encoder.layer.8.output.dense', 
'bert.encoder.layer.9.output.dense'], params={'num_terms': [1, 2, 3], 'low_rank': [256, 32, 64, 128, 16]}, 
separate=False, copy_weights=True), LayerConfig(layer_names=['cls.predictions.decoder'], params={'num_terms': [1, 
2, 3], 'low_rank': [256, 32, 64, 128, 16]}, separate=False, copy_weights=True)])

## 6. Run Optimization
Executing the search for best parameters and applying them.

In [8]:
best_params = tuner.tune()
tuned_model = tuner.apply_best_params()

Trying parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.0.attention.output.dense with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.0.attention.self.key with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.0.attention.self.query with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.0.attention.self.value with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.1.attention.output.dense with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.1.attention.self.key with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.1.attention.self.query with sketched version using parameters: {'num_terms': 1, 'low_rank': 16}
Replaced bert.encoder.layer.1.attention.self.value with sketched version using p

D:\Coding Projects\panther\panther\tuner\SkAutoTuner\SKAutoTuner.py:263: UserWarning: The sketching layer uses more parameters than a fully connected layer. Consider reducing 'num_terms' or 'low_rank' for efficiency.
  sketched_layer = sketched_class(


Replaced bert.encoder.layer.3.attention.self.query with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.3.attention.self.value with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.4.attention.output.dense with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.4.attention.self.key with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.4.attention.self.query with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.4.attention.self.value with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.5.attention.output.dense with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}
Replaced bert.encoder.layer.5.attention.self.key with sketched version using parameters: {'num_terms': 2, 'low_rank': 128}


## 7. Final Results & Comparison
Comparing the Tuned model vs. Original model under identical conditions.

In [9]:
# Standardized test function
def test_model(model_name, model):
    model.eval()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    results = evaluate_model_with_stats(model, val_loader, tokenizer, n_runs=5)

    def infer(model, inputs):
        with torch.no_grad():
            return model(**inputs)

    time_result = measure_time_with_stats(infer, model, memory_batch, n_runs=5)

    return {
        "name": model_name,
        "loss": results["loss_mean"],
        "loss_std": results["loss_std"],
        "speed": time_result["samples_per_sec"],
        "speed_std": time_result["samples_per_sec_std"],
    }


# Run tests
baseline_results = test_model("Original", orig_model)
tuned_results = test_model("Tuned", tuned_model)

# Parameter reduction stats
tuned_params = model_size_info(tuned_model)
reduction_pct = (
    1 - tuned_params["total_params_millions"] / orig_params["total_params_millions"]
) * 100

print(f"\nOriginal params: {orig_params['total_params_millions']:.2f}M")
print(f"Tuned params: {tuned_params['total_params_millions']:.2f}M")
print(f"Reduction: {reduction_pct:.2f}%")

# Print Table
# Define column widths for consistent alignment
col_fmt = "| {:<15} | {:<20} | {:<25} | {:<20} |"
divider = "-" * 91

print("\n" + "=" * 91)
print(f"{'PERFORMANCE COMPARISON':^91}")
print("=" * 91)
print(col_fmt.format("Model Version", "MLM Loss", "Speed (samples/sec)", "Improvement"))
print(divider)

# Pre-format values to ensure perfect alignment within the columns
# Original Model Data
orig_loss = f"{baseline_results['loss']:.4f} ± {baseline_results['loss_std']:.4f}"
orig_speed = f"{baseline_results['speed']:.2f} ± {baseline_results['speed_std']:.2f}"
print(col_fmt.format("Original", orig_loss, orig_speed, "1.00x"))

# Tuned Model Data
tuned_loss = f"{tuned_results['loss']:.4f} ± {tuned_results['loss_std']:.4f}"
tuned_speed = f"{tuned_results['speed']:.2f} ± {tuned_results['speed_std']:.2f}"
speed_up = f"{tuned_results['speed'] / baseline_results['speed']:.2f}x"
print(col_fmt.format("Tuned", tuned_loss, tuned_speed, speed_up))

print(divider)

Original params: 109.51M

Tuned params: 27.20M

Reduction: 75.16%

===========================================================================================

PERFORMANCE COMPARISON

...

| Model Version   | MLM Loss             | Speed (samples/sec)       | Improvement          |

-------------------------------------------------------------------------------------------

| Original        | 4.6011 ± 0.0041      | 15.48 ± 0.11              | 1.00x                |

| Tuned           | 4.5940 ± 0.0061      | 14.90 ± 0.16              | 0.96x                |

-------------------------------------------------------------------------------------------